<a href="https://colab.research.google.com/github/abhishekmulik/LP5/blob/main/HPC_PRAC_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile HPC_2A_Parallel_Bubble_Sort.cpp
#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

void bubble_sort_odd_even(vector<int>& arr) {
	bool isSorted = false;
	while(!isSorted) {
		isSorted = true;
		
		#pragma omp parallel for
		for(int i = 0; i < arr.size()-1; i+= 2){
			if(arr[i] > arr[i+1]){
				swap(arr[i],arr[i+1]);
				isSorted = false;
			}
		}
		#pragma omp parallel for
		for(int i = 1; i < arr.size()-1; i += 2) {
			if(arr[i] > arr[i + 1]) {
				swap(arr[i],arr[i+1]);
				isSorted = false;
			}
		}
	}
}

void print(vector<int> const &arr)
{
    
    for (int i = 0; i < arr.size(); i++) {
        cout << arr.at(i) << ' ';
    }
    cout<<endl;
}

int main() {
	vector<int> arr = {5, 2, 9, 1, 7, 6, 8, 3, 4};
	double start, end;
	cout<<"Original Input Before Sorting:";
	print(arr);
// Measure performance of parallel bubble sort using odd-even transposition
	start = omp_get_wtime();
	bubble_sort_odd_even(arr);
	end = omp_get_wtime();
	cout<<"After Sorting:";
	print(arr);
	cout << "Parallel bubble sort using odd-even transposition time: "<<end-start << endl;
	
}


Overwriting HPC_2A_Parallel_Bubble_Sort.cpp


In [ ]:
! g++ -fopenmp HPC_2A_Parallel_Bubble_Sort.cpp -o bubblesort

In [ ]:
!./bubblesort

Original Input Before Sorting:5 2 9 1 7 6 8 3 4 
After Sorting:1 2 3 4 5 6 7 8 9 
Parallel bubble sort using odd-even transposition time: 0.0143159


In [ ]:
%%writefile HPC_2A_Parallel_Bubble_Sort.cpp
#include <iostream>
#include <chrono>
#include <omp.h>
#include <stdio.h>
#include <random>
#include <string.h>
#include <stdlib.h>

using namespace std;
using namespace std::chrono;

int random_in_range( int minimum, int maximum )
{
  thread_local std::ranlux48 rng( 
    std::chrono::system_clock::now().time_since_epoch().count() );
  return std::uniform_int_distribution <int> ( minimum, maximum )( rng );
}

void merge(int *c1, int *c2, int *p, int cl1, int cl2) {
	int i = 0, j = 0, k = 0;
	while(i < cl1 && j < cl2) {
		if(c1[i] < c2[j]) {
			p[k] = c1[i];
			i++;
			k++;
		}
		else {
			p[k] = c2[j];
			j++;
			k++;
		}
	}
	while(i < cl1) {
		p[k] = c1[i];
		i++;
		k++;
	}
	while(j < cl2) {
		p[k] = c2[j];
		j++;
		k++;
	}
}

void mergeSortSerial(int *arr, int length) {
	if(length <= 1)
		return;

	int mid = length / 2;
	
	int *left = new int[mid];
	int *right = new int[length-mid];

	for(int i = 0 ; i < mid ; i++) {
		left[i] = arr[i];
	}
	for(int i = mid ; i < length ; i++) {
		right[i-mid] = arr[i];
	}
	mergeSortSerial(left, mid);
	mergeSortSerial(right, length-mid);
	merge(left, right, arr, mid, length-mid);
}

void mergeSortParallel(int *arr, int length) {
	if(length <= 1)
		return;
	int mid = length / 2;
	int *left, *right;
	left = new int[mid];
	right = new int[length-mid];

	#pragma omp task firstprivate(left)
	{
		
		for(int i = 0 ; i < mid ; i++) {
			left[i] = arr[i];
		}
		mergeSortParallel(left, mid);
	}
	
	
	#pragma omp task firstprivate(right)
	{
		
		for(int i = mid ; i < length ; i++) {
			right[i-mid] = arr[i];
		}
	
		mergeSortParallel(right, length-mid);		
	}


	// #pragma omp parallel sections
	// {
	// 	#pragma omp section
	// 	{
	// 		left = new int[mid];
	// 		for(int i = 0 ; i < mid ; i++) {
	// 			left[i] = arr[i];
	// 		}
	// 		mergeSortParallel(left, mid);
	// 	}
	// 	#pragma omp section
	// 	{
	// 		right = new int[length-mid];
	// 		for(int i = mid ; i < length ; i++) {
	// 			right[i-mid] = arr[i];
	// 		}
	// 		mergeSortParallel(right, length-mid);
	// 	}
	// }
	#pragma omp taskwait
	merge(left, right, arr, mid, length-mid);
}

void init_array(int *arr1, int *arr2, int n) {
	for(int i = 0 ; i < n ; i++) {
		arr1[i] = arr2[i] = random_in_range(10,99);
	}
}

void print_array(int *arr, int n) {
	for(int i = 0 ; i < n ; i++) {
		cout<<arr[i]<<" ";
	}
}

int main() {
	int n = 50;
	int *a, *b;
	a = new int[n];
	b = new int[n];

	init_array(a, b, n);
	cout<<"Initial vector: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;

	time_point<system_clock> starttime, endtime;
	starttime = system_clock::now();
	mergeSortSerial(a, n);
	endtime = system_clock::now();
	duration<double> time = endtime - starttime;
	cout<<"Time for serial: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after serial merge sort: "<<endl;
	print_array(a, n);
	cout<<endl;
	cout<<endl;


	starttime = system_clock::now();
	mergeSortParallel(b, n);
	endtime = system_clock::now();
	time = endtime - starttime;
	cout<<"Time for parallel: "<<1000*time.count()<<" milliseconds"<<endl;
	cout<<"Result after parallel merge sort: "<<endl;
	print_array(b, n);

	return 0;	
}


Overwriting HPC_2A_Parallel_Bubble_Sort.cpp


In [ ]:
! g++ -fopenmp MergeSort_Algorithms.cpp -o Mergesort

In [ ]:
!./Mergesort

Initial vector: 
47 43 39 24 69 79 39 63 84 84 69 12 94 75 77 17 34 16 91 52 36 60 84 58 33 90 98 67 29 78 91 58 71 76 95 32 13 38 34 39 53 61 57 96 68 76 74 49 21 86 

Time for serial: 0.015704 milliseconds
Result after serial merge sort: 
12 13 16 17 21 24 29 32 33 34 34 36 38 39 39 39 43 47 49 52 53 57 58 58 60 61 63 67 68 69 69 71 74 75 76 76 77 78 79 84 84 84 86 90 91 91 94 95 96 98 

Time for parallel: 0.017185 milliseconds
Result after parallel merge sort: 
12 13 16 17 21 24 29 32 33 34 34 36 38 39 39 39 43 47 49 52 53 57 58 58 60 61 63 67 68 69 69 71 74 75 76 76 77 78 79 84 84 84 86 90 91 91 94 95 96 98 